In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, linear_reset
from statsmodels.stats.diagnostic import acorr_breusch_godfrey

import os
from pathlib import Path

In [95]:
DATA_DIR = Path("../../../data/")
DATA_DIR.mkdir(parents=True, exist_ok=True)

LATEX_OUT = Path("../../../docs/latex_utils/tables")
LATEX_OUT.mkdir(parents=True, exist_ok=True)

PLOTS_DIR = Path("../../../plots/python/ex8/")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

In [96]:
def save_plot(
    plot: plt.Figure,
    filename: str,
    format: str = "png",
    dpi: int = 300,
    close: bool = True,
):
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)
    filepath = PLOTS_DIR / f"{filename}.{format}"
    try:
        plot.savefig(filepath, format=format, dpi=dpi, bbox_inches="tight")
        if close:
            plt.close(plot)
        print(
            f"\nPlot {filename}.{format} saved correctly in {PLOTS_DIR}/{filename}.{format}"
        )
    except Exception as e:
        print(f"\nCould not save plot {filename}.{format}. Reason: {e}")

In [97]:
def save_latex_table(df, filename: str, rename_map: dict, caption: str, label: str):
    try:
        table_tex = df.rename(columns=rename_map).to_latex(
            index=False,
            float_format="%.4f",
            caption=caption,
            label=label,
        )
        with open(LATEX_OUT / filename, "w") as f:
            f.write(table_tex)
        print(f"\nFile {filename} exported correctly in {LATEX_OUT}/{filename}")
    except Exception as e:
        print(f"\nCould not export {filename}. Reason: {e}")

In [98]:
df = pd.read_excel(os.path.join(DATA_DIR, "Table12_9.xls"))
print(df.head())

   Year  Sales  Inventories     Ratio
0  1950  46486        84646  1.820892
1  1951  50229        90560  1.802943
2  1952  53501        98145  1.834452
3  1953  52805       101599  1.924041
4  1954  55906       102567  1.834633


In [99]:
cols_map = {
    "Year": "year",
    "Sales": "sales",
    "Inventories": "inventories",
    "Ratio": "ratio",
}
df = df.rename(columns=cols_map)

numeric_cols = ["year", "sales", "inventories", "ratio"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
df = df.sort_values("year").reset_index(drop=True)

print("\nDimensiones:", df.shape)
print("\nDescripción estadística básica:\n", df.describe().T)


Dimensiones: (41, 4)

Descripción estadística básica:
              count           mean            std          min           25%  \
year          41.0    1970.000000      11.979149   1950.00000    1960.00000   
sales         41.0  208038.658537  107874.906465  46486.00000  113201.00000   
inventories   41.0  312958.121951  131513.466995  84646.00000  188378.00000   
ratio         41.0       1.600934       0.208567      1.21411       1.43618   

                       50%            75%            max  
year           1970.000000    1980.000000    1990.000000  
sales        206326.000000  299766.000000  411663.000000  
inventories  339516.000000  423082.000000  509902.000000  
ratio             1.657551       1.767625       1.924041  


In [100]:
desc = (
    df[["sales", "inventories", "ratio"]]
    .describe()
    .T.reset_index()
    .rename(columns={"index": "variable"})
)
save_latex_table(
    desc,
    filename="ex8_descriptivos.tex",
    rename_map={
        "variable": "Variable",
        "count": "N",
        "mean": "Media",
        "std": "Desv.Est.",
        "min": "Mínimo",
        "25%": "P25",
        "50%": "P50",
        "75%": "P75",
        "max": "Máximo",
    },
    caption="Estadísticos descriptivos de ventas, inventarios y razón inventarios/ventas.",
    label="tab:ex8_desc",
)


File ex8_descriptivos.tex exported correctly in ../../../docs/latex_utils/tables/ex8_descriptivos.tex


In [101]:
corr = (
    df[["sales", "inventories", "ratio"]]
    .corr()
    .reset_index()
    .rename(columns={"index": "variable"})
)
save_latex_table(
    corr,
    filename="ex8_correlaciones.tex",
    rename_map={
        "variable": "Variable",
        "sales": "Ventas",
        "inventories": "Inventarios",
        "ratio": "Razón",
    },
    caption="Matriz de correlaciones.",
    label="tab:ex8_corr",
)


File ex8_correlaciones.tex exported correctly in ../../../docs/latex_utils/tables/ex8_correlaciones.tex


In [102]:
plt.figure()
plt.plot(df["year"], df["sales"], marker="o", label="Ventas")
plt.plot(df["year"], df["inventories"], marker="s", label="Inventarios")
plt.xlabel("Año")
plt.ylabel("Nivel")
plt.title("Series en el tiempo: Ventas e Inventarios")
plt.legend()
save_plot(plt.gcf(), filename="ex8_series_ventas_inventarios")


Plot ex8_series_ventas_inventarios.png saved correctly in ../../../plots/python/ex8/ex8_series_ventas_inventarios.png


In [103]:
plt.figure()
plt.plot(df["year"], df["ratio"], marker="o")
plt.xlabel("Año")
plt.ylabel("Inventarios/Ventas")
plt.title("Razón Inventarios/Ventas en el tiempo")
save_plot(plt.gcf(), filename="ex8_series_ratio")


Plot ex8_series_ratio.png saved correctly in ../../../plots/python/ex8/ex8_series_ratio.png


In [104]:
plt.figure()
plt.scatter(df["sales"], df["inventories"])
b1, b0, r, p, se = stats.linregress(df["sales"], df["inventories"])
xline = np.linspace(df["sales"].min(), df["sales"].max(), 100)
plt.plot(xline, b0 + b1 * xline)
plt.xlabel("Ventas")
plt.ylabel("Inventarios")
plt.title("Inventarios vs. Ventas (con línea OLS)")
save_plot(plt.gcf(), filename="ex8_scatter_inv_vs_sales")


Plot ex8_scatter_inv_vs_sales.png saved correctly in ../../../plots/python/ex8/ex8_scatter_inv_vs_sales.png


In [105]:
X1 = sm.add_constant(df[["sales"]])
model1 = sm.OLS(df["inventories"], X1).fit()
print("\n=== Modelo 1: Inventarios ~ Ventas ===\n", model1.summary())


=== Modelo 1: Inventarios ~ Ventas ===
                             OLS Regression Results                            
Dep. Variable:            inventories   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     645.0
Date:                Sat, 20 Sep 2025   Prob (F-statistic):           7.21e-26
Time:                        23:14:32   Log-Likelihood:                -482.21
No. Observations:                  41   AIC:                             968.4
Df Residuals:                      39   BIC:                             971.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      

In [106]:
X2 = sm.add_constant(df[["sales", "year"]])
model2 = sm.OLS(df["inventories"], X2).fit()
print("\n=== Modelo 2: Inventarios ~ Ventas + Año ===\n", model2.summary())


=== Modelo 2: Inventarios ~ Ventas + Año ===
                             OLS Regression Results                            
Dep. Variable:            inventories   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     320.1
Date:                Sat, 20 Sep 2025   Prob (F-statistic):           1.66e-24
Time:                        23:14:32   Log-Likelihood:                -481.85
No. Observations:                  41   AIC:                             969.7
Df Residuals:                      38   BIC:                             974.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const

In [107]:
def _params_to_df(res):
    out = (
        res.params.to_frame("coef")
        .join(res.bse.to_frame("se"))
        .join(res.tvalues.to_frame("t"))
        .join(res.pvalues.to_frame("p"))
    )
    ci = res.conf_int()
    ci.columns = ["ci_low", "ci_high"]
    out = out.join(ci)
    out.index.name = "Parametro"
    return out.reset_index()

In [108]:
coef1 = _params_to_df(model1)
coef2 = _params_to_df(model2)

save_latex_table(
    coef1,
    filename="ex8_ols_m1.tex",
    rename_map={
        "Parametro": "Parámetro",
        "coef": "Coef.",
        "se": "E.E.",
        "t": "t",
        "p": "p-valor",
        "ci_low": "IC 2.5%",
        "ci_high": "IC 97.5%",
    },
    caption="Resultados OLS: Inventarios sobre Ventas.",
    label="tab:ex8_m1",
)


File ex8_ols_m1.tex exported correctly in ../../../docs/latex_utils/tables/ex8_ols_m1.tex


In [109]:
save_latex_table(
    coef2,
    filename="ex8_ols_m2.tex",
    rename_map={
        "Parametro": "Parámetro",
        "coef": "Coef.",
        "se": "E.E.",
        "t": "t",
        "p": "p-valor",
        "ci_low": "IC 2.5%",
        "ci_high": "IC 97.5%",
    },
    caption="Resultados OLS: Inventarios sobre Ventas y Año.",
    label="tab:ex8_m2",
)


File ex8_ols_m2.tex exported correctly in ../../../docs/latex_utils/tables/ex8_ols_m2.tex


In [110]:
res1 = model1.resid
res2 = model2.resid

fig = qqplot(res1, line="45")
plt.title("Q-Q plot residuos Modelo 1")
save_plot(fig.figure, filename="ex8_qq_m1")


Plot ex8_qq_m1.png saved correctly in ../../../plots/python/ex8/ex8_qq_m1.png


In [111]:
fig = qqplot(res2, line="45")
plt.title("Q-Q plot residuos Modelo 2")
save_plot(fig.figure, filename="ex8_qq_m2")


Plot ex8_qq_m2.png saved correctly in ../../../plots/python/ex8/ex8_qq_m2.png


In [112]:
fig = plot_acf(res1, lags=12)
plt.title("ACF residuos Modelo 1")
save_plot(fig.figure, filename="ex8_acf_m1")


Plot ex8_acf_m1.png saved correctly in ../../../plots/python/ex8/ex8_acf_m1.png


In [113]:
fig = plot_acf(res2, lags=12)
plt.title("ACF residuos Modelo 2")
save_plot(fig.figure, filename="ex8_acf_m2")


Plot ex8_acf_m2.png saved correctly in ../../../plots/python/ex8/ex8_acf_m2.png


In [114]:
dw1 = sm.stats.stattools.durbin_watson(res1)
dw2 = sm.stats.stattools.durbin_watson(res2)
print(f"\nDurbin-Watson M1: {dw1:.4f}  |  M2: {dw2:.4f}")


Durbin-Watson M1: 0.1256  |  M2: 0.1254


In [115]:
bp1 = het_breuschpagan(res1, X1)  # (LM stat, LM pvalue, F stat, F pvalue)
bp2 = het_breuschpagan(res2, X2)
print("\nBreusch-Pagan M1 (LM, p, F, p):", tuple(round(x, 4) for x in bp1))
print("Breusch-Pagan M2 (LM, p, F, p):", tuple(round(x, 4) for x in bp2))


Breusch-Pagan M1 (LM, p, F, p): (np.float64(1.7818), np.float64(0.1819), np.float64(1.7719), np.float64(0.1909))
Breusch-Pagan M2 (LM, p, F, p): (np.float64(3.0968), np.float64(0.2126), np.float64(1.5523), np.float64(0.2249))


In [116]:
wt1 = het_white(res1, X1)
wt2 = het_white(res2, X2)
print("\nWhite test M1 (LM, p, F, p):", tuple(round(x, 4) for x in wt1))
print("White test M2 (LM, p, F, p):", tuple(round(x, 4) for x in wt2))


White test M1 (LM, p, F, p): (np.float64(2.8789), np.float64(0.2371), np.float64(1.4349), np.float64(0.2508))
White test M2 (LM, p, F, p): (np.float64(6.7043), np.float64(0.1524), np.float64(1.7594), np.float64(0.1585))


In [117]:
jb1 = sm.stats.stattools.jarque_bera(res1)
jb2 = sm.stats.stattools.jarque_bera(res2)
print("\nJarque-Bera M1 (JB, p, skew, kurt):", tuple(round(float(x), 4) for x in jb1))
print("Jarque-Bera M2 (JB, p, skew, kurt):", tuple(round(float(x), 4) for x in jb2))


Jarque-Bera M1 (JB, p, skew, kurt): (2.0457, 0.3596, -0.0515, 1.9106)
Jarque-Bera M2 (JB, p, skew, kurt): (1.994, 0.369, -0.0987, 1.9378)


In [118]:
reset1 = linear_reset(model1, use_f=True)
reset2 = linear_reset(model2, use_f=True)
print(
    "\nRESET Ramsey M1 (F, p, gl):",
    round(reset1.fvalue, 4),
    round(reset1.pvalue, 4),
    reset1.df_denom,
)
print(
    "RESET Ramsey M2 (F, p, gl):",
    round(reset2.fvalue, 4),
    round(reset2.pvalue, 4),
    reset2.df_denom,
)


RESET Ramsey M1 (F, p, gl): 138.0126 0.0 38.0
RESET Ramsey M2 (F, p, gl): 32.471 0.0 37.0


In [ ]:
bg1_lm, bg1_lm_p, bg1_f, bg1_f_p = acorr_breusch_godfrey(model1, nlags=1)
bg2_lm, bg2_lm_p, bg2_f, bg2_f_p = acorr_breusch_godfrey(model2, nlags=1)

print(
    f"\nBreusch–Godfrey M1 (LM={bg1_lm:.4f}, p={bg1_lm_p:.4g}; F={bg1_f:.4f}, p={bg1_f_p:.4g})"
)
print(
    f"Breusch–Godfrey M2 (LM={bg2_lm:.4f}, p={bg2_lm_p:.4g}; F={bg2_f:.4f}, p={bg2_f_p:.4g})"
)

In [ ]:
idx = ["M1: Inv ~ Ventas", "M2: Inv ~ Ventas + Año"]
results_tests = (
    pd.DataFrame(
        {
            "Durbin-Watson": [dw1, dw2],
            "BP p-valor": [bp1[1], bp2[1]],
            "White p-valor": [wt1[1], wt2[1]],
            "JB p-valor": [float(jb1[1]), float(jb2[1])],
            "RESET p-valor": [reset1.pvalue, reset2.pvalue],
            "BG LM p-valor": [bg1_lm_p, bg2_lm_p],
            "BG F p-valor": [bg1_f_p, bg2_f_p],
            "R2": [model1.rsquared, model2.rsquared],
            "R2 Ajustado": [model1.rsquared_adj, model2.rsquared_adj],
            "N": [int(model1.nobs), int(model2.nobs)],
        },
        index=["M1: Inv ~ Ventas", "M2: Inv ~ Ventas + Año"],
    )
    .reset_index()
    .rename(columns={"index": "Modelo"})
)


save_latex_table(
    results_tests.round(4),
    filename="ex8_diagnosticos.tex",
    rename_map={
        "Modelo": "Modelo",
        "Durbin-Watson": "Durbin–Watson",
        "BP p-valor": "Breusch–Pagan p",
        "White p-valor": "White p",
        "JB p-valor": "Jarque–Bera p",
        "RESET p-valor": "RESET p",
        "R2": "$R^2$",
        "R2 Ajustado": "$R^2$ Ajustado",
        "N": "N",
    },
    caption="Pruebas de supuestos y métricas de ajuste para los modelos estimados.",
    label="tab:ex8_diagnosticos",
)


File ex8_diagnosticos.tex exported correctly in ../../../docs/latex_utils/tables/ex8_diagnosticos.tex


In [120]:
plt.figure()
plt.plot(df["year"], res1, marker="o")
plt.axhline(0, linestyle="--")
plt.xlabel("Año")
plt.ylabel("Residuo")
plt.title("Residuos en el tiempo – Modelo 1")
save_plot(plt.gcf(), filename="ex8_residuos_m1")


Plot ex8_residuos_m1.png saved correctly in ../../../plots/python/ex8/ex8_residuos_m1.png


In [121]:
plt.figure()
plt.plot(df["year"], res2, marker="o")
plt.axhline(0, linestyle="--")
plt.xlabel("Año")
plt.ylabel("Residuo")
plt.title("Residuos en el tiempo – Modelo 2")
save_plot(plt.gcf(), filename="ex8_residuos_m2")


Plot ex8_residuos_m2.png saved correctly in ../../../plots/python/ex8/ex8_residuos_m2.png
